In [1]:
import pymc3 as pm

# Some PyMC3 basics

A PyMC model is defined by relating unknown variables to observed data. Think of it as associating the observed data with a proposal for the data generation scheme that led to it.

## Model Context

PyMC3 uses python's `with...as...:` syntax to create a model context. It's a bit like `using(...) {...}` in C#.

```
with EXPRESSION as BLAH:
    SOME CODE
```

...is equivalent to...

```
BLAH = EXPRESSION
BLAH.__enter__()
try:
    SOME CODE
finally:
    BLAH.__exit__()
```

In PyMC3 any variables created in the context of a new model (`with pm.Model() as model:`) or an existing model (`with existing_model:`) are automatically associated with the model. You get an error if you create a PyMC variable outside the context of a model.

## Variables

PyMC3 variables have an initial value used as the starting point of the sampling. `tag.test_value` can be used to query the starting point. You can specify a particular starting value using the `testval` parameter...

In [41]:
with pm.Model() as model:
    var1 = pm.Exponential('var1', 3)
    var2 = pm.Exponential('var2', 3, testval=0.4)
    
print(var1.tag.test_value)
print(var2.tag.test_value)

0.2310490608215332
0.4


PyMC3 variables can be *Stochastic* or *Deterministic*...

* *Stochastic:* even if we know all of its input parameters we still don't know its value, i.e., it's still a random variable. For example, a Poisson random variable - even if know the value of its parameter $\lambda$ it's still a random variable with a range of possible values. Other examples are Discrete Uniform and Exponential random variables (in fact, any of the probability distributions we've already discussed - and many we haven't).
* *Deterministic:* its value can be determined exactly if we know all of its input parameters.

Stochastic variables are created with a name argument of type string and other parameters specific to each type of variable. For example...

`some_variable = pm.DiscreteUniform("discrete_uni_var", 0, 4)`

...where $0$ and $4$ are the upper & lower bound parameters for the discrete uniform distribution.

Note that the name string is used as a key for the [backend](https://docs.pymc.io/api/backends.html) and for display purposes in various plots, graphs & traces you can get out of PyMC3. It's usual to use the name of the object variable for the name argument - but it doesn't have to be!

For convenience, the `shape` parameter allows you to create an array of independent stochastic variables instead of creating arbitrary names and variables for each one. For example, instead of...


    beta_1 = pm.Uniform("beta_1", 0, 1)
    beta_2 = pm.Uniform("beta_2", 0, 1)
    ...
    
...use a single variable...

    betas = pm.Uniform("betas", 0, 1, shape=N)

In [44]:
with pm.Model() as model:
     betas = pm.Uniform("betas", 0, 1, shape=4)

betas.tag.test_value

array([0.5, 0.5, 0.5, 0.5])